# Introduction to Python Classes

In [44]:
import numpy as np

Object-oriented programming often provides a clean, easy to remember, API to your functions that you often use. Let's see if this is the case for machine learning models.

The main steps we have been taking to build, train and use our simple neural networks are
1. Create the weight matrices.
2. Train our neural networks by repeatedly updating the weight matrices using stochastic gradient descent.
3. Use, or apply, our trained neural networks on new data.

Humm.....what shall we "name" these steps?  If we define a neural network as an object, or instance, of class `NeuralNetwork`, then we can call these steps

2. train
3. use

and the creation of the weight matrices can be part of the constructor for the `NeuralNetwork` object.

Okay. Next step is to design our API, meaning define the arguments each step takes.

How about this example python code?
```
nnet = NeuralNetwork(...)
nnet.train(Xtrain, Ttrain, nepochs=1000, learning_rate=0.01)
Ytest = nnet.use(Xtest)
```

But what arguments should be pass to the constructor?  To define the weight matrices, we need to know the number of input components (features), the number of hidden layers and the number of units in each hidden layer, and the number of outputs. We can specify both the number of hidden layers and the number of units in each hidden layer with a list of integers.
```
nnet = NeuralNetwork(n_inputs, n_hiddens_each_layer, n_outputs)
```
So, to define a neural network to be trained on data with 5 values in each input sample, targets of 3 values in each sample, and having 3 hidden layers with 20, 10, and 5 units each, we could call
```
nnet = NeuralNetwork(5, [20, 10, 5], 3)
```

Okay, let's get to coding, taking baby steps for those of us who have never defined a Python class.

Here is the smallest class we can define.

In [45]:
class NeuralNetwork:
    pass

How would you use this class?

In [46]:
nnet = NeuralNetwork()
nnet

In [47]:
type(nnet)

__main__.NeuralNetwork

Well, that's rather cryptic.  Let's define a printed representation of our object, much like you would define `toString` in Java.

In [48]:
class NeuralNetwork:
    
    def __str__(self):
        return 'NeuralNetwork()'

In [49]:
nnet = NeuralNetwork()
nnet

That didn't work.  The `__str__` method is actually meant for the printed representation.

In [50]:
print(nnet)

NeuralNetwork()


The string that results from evaluating a variable is meant to be a representation as close to the python code as we can get that was called to create the object. It is returned by `__repr__`. Turns out to be the same in this simple case.

In [51]:
class NeuralNetwork:
    
    def __repr__(self):
        return 'NeuralNetwork()'
    
    def __str__(self):
        return 'NeuralNetwork()'

In [52]:
nnet = NeuralNetwork()
nnet

NeuralNetwork()

We will probably want to provide a bit more information in the result of `__str__`.  Maybe something like

In [53]:
class NeuralNetwork:
    
    def __repr__(self):
        return 'NeuralNetwork()'
    
    def __str__(self):
        return self.__repr__() + ', trained for 100 epochs with a final RMSE of 0.2'

In [54]:
nnet = NeuralNetwork()
print(nnet)
nnet

NeuralNetwork(), trained for 100 epochs with a final RMSE of 0.2


NeuralNetwork()

Let's tackle the constructor now. It also has a double-underscore prefix and postfix. In the constructor, let's save the arguments in member variables, and create the weight matrices as member variables. 

To allow ourselves to create neural networks with any number of hidden layers, let's create a list of weight matrices with one weight matrix for every layer. In the following code, we are hardcoding for just two hidden layers.  You will modify this in the next assignment to construct as many weight matrices as you have integers in `n_hiddens_each_layer`.

Now we should modify `__repr__` to show the full python call that would create our object.

In [55]:
class NeuralNetwork:
    
    def __init__(self, n_inputs, n_hiddens_each_layer, n_outputs):
        
        self.n_inputs = n_inputs
        self.n_hiddens_each_layer = n_hiddens_each_layer
        self.n_outputs = n_outputs

        self.n_epochs = None
        self.rmse = None
               
        self.W = []
        ni = self.n_inputs
        nh = self.n_hiddens_each_layer[0]
        self.W.append(np.random.uniform(-1, 1, size=(1 + ni, nh)) / np.sqrt(1 + ni))
        
        ni = nh
        nh = self.n_hiddens_each_layer[1]
        self.W.append(np.random.uniform(-1, 1, size=(1 + ni, nh)) / np.sqrt(1 + ni))
    
    def __repr__(self):
        return 'NeuralNetwork({}, {}, {})'.format(self.n_inputs, self.n_hiddens_each_layer, self.n_outputs)
    
    def __str__(self):
        return self.__repr__() + ', trained for {} epochs with a final RMSE of {}'.format(self.n_epochs, self.rmse)


In [56]:
nnet = NeuralNetwork(5, [10, 5], 3)
nnet

NeuralNetwork(5, [10, 5], 3)

In [57]:
print(nnet)

NeuralNetwork(5, [10, 5], 3), trained for None epochs with a final RMSE of None


We can examine the values of any member variable directly.  Nothing private or protected in python!

In [58]:
nnet.n_outputs

3

In [59]:
len(nnet.W)

2

Now we can add "empty" member functions for our remaining two functions.  

If you wish to define additional functions that your member functions will call, it is convention to prefix them with a single underscore.

Don't forget that first argument `self` !

In [60]:
class NeuralNetwork:
    
    def __init__(self, n_inputs, n_hiddens_each_layer, n_outputs):
        
        self.n_inputs = n_inputs
        self.n_hiddens_each_layer = n_hiddens_each_layer
        self.n_outputs = n_outputs
        
        self.n_epochs = None
        self.rmse = None
        
        self.W = []
        
        # Hidden Layer 1
        ni = self.n_inputs
        nu = self.n_hiddens_each_layer[0]
        self.W.append(np.random.uniform(-1, 1, size=(1 + ni, nu)) / np.sqrt(1 + ni))
        
        # Hidden Layer 2
        ni = nu
        nu = self.n_hiddens_each_layer[1]
        self.W.append(np.random.uniform(-1, 1, size=(1 + ni, nu)) / np.sqrt(1 + ni))
        
        # Output Layer
        ni = nu
        nu = self.n_outputs
        self.W.append(np.random.uniform(-1, 1, size=(1 + ni, nu)) / np.sqrt(1 + ni))
        
    
    def __repr__(self):
        return 'NeuralNetwork({}, {}, {})'.format(self.n_inputs, self.n_hiddens_each_layer, self.n_outputs)
    
    def __str__(self):
        return self.__repr__() + ', trained for {} epochs with a final RMSE of {}'.format(self.n_epochs, self.rmse)
    
    def train(self, X, T, n_epochs, learning_rate):
        pass
    
    def use(self, X):
        pass
    
    def _forward(self, X):
        pass
    
    def _gradient(self, X, T):
        pass

In [61]:
nnet = NeuralNetwork(2, [10, 3], 1)
print(nnet)

NeuralNetwork(2, [10, 3], 1), trained for None epochs with a final RMSE of None


That's it!   Now you can get to work on Assignment A2!